In [ ]:
# Встановлення необхідних бібліотек
!apt update &> /dev/null
!apt install ffmpeg -y
!pip install gdown

import os
import subprocess
import gdown
from google.colab import drive, files
import json
import re


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
# Підключаємо Google Drive
from google.colab import drive
drive.mount('/content/drive')

print("✅ Google Drive підключено успішно!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive підключено успішно!


In [ ]:
def download_video_from_url(url, output_filename):
    """
    Завантажує відео з прямого посилання
    """
    try:
        print(f"Завантаження відео: {output_filename}")

        # Використовуємо wget для завантаження
        result = subprocess.run([
            'wget', '-O', output_filename, url, '--no-check-certificate'
        ], capture_output=True, text=True)

        if result.returncode == 0 and os.path.exists(output_filename):
            print(f"✅ Відео завантажено: {output_filename}")
            return output_filename
        else:
            # Пробуємо curl якщо wget не спрацював
            result = subprocess.run([
                'curl', '-L', '-o', output_filename, url
            ], capture_output=True, text=True)

            if result.returncode == 0 and os.path.exists(output_filename):
                print(f"✅ Відео завантажено через curl: {output_filename}")
                return output_filename
            else:
                print(f"❌ Помилка завантаження: {result.stderr}")
                return None

    except Exception as e:
        print(f"❌ Помилка: {e}")
        return None

def get_video_info(video_path):
    """
    Отримує інформацію про відео за допомогою ffprobe
    """
    try:
        cmd = [
            'ffprobe', '-v', 'quiet', '-print_format', 'json',
            '-show_streams', '-show_format', video_path
        ]
        result = subprocess.run(cmd, capture_output=True, text=True)

        if result.returncode == 0:
            info = json.loads(result.stdout)

            # Знаходимо відео стрім
            video_stream = None
            audio_stream = None

            for stream in info['streams']:
                if stream['codec_type'] == 'video' and video_stream is None:
                    video_stream = stream
                elif stream['codec_type'] == 'audio' and audio_stream is None:
                    audio_stream = stream

            return {
                'duration': float(info['format']['duration']),
                'has_audio': audio_stream is not None,
                'width': video_stream['width'] if video_stream else 0,
                'height': video_stream['height'] if video_stream else 0,
                'fps': eval(video_stream['r_frame_rate']) if video_stream else 0
            }
        else:
            print(f"Помилка отримання інформації: {result.stderr}")
            return None

    except Exception as e:
        print(f"Помилка: {e}")
        return None


In [ ]:
def add_hardcoded_subtitles(video_path, subtitles_path, output_path, subtitle_style=None):
    """
    Додає жорсткі субтитри до відео за допомогою FFmpeg
    """
    try:
        # Базовий стиль субтитрів
        if subtitle_style is None:
            subtitle_style = {
                'FontName': 'Arial',
                'FontSize': '12',
                'PrimaryColour': '&Hffffff',  # Білий колір
                'BackColour': '&H80000000',   # Напівпрозорий чорний фон
                'Bold': '1',
                'BorderStyle': '1',
                'Outline': '2',
                'Shadow': '0',
                'Alignment': '2',  # По центру знизу
                'MarginV': '30'
            }

        print(f"🎬 Додавання субтитрів до відео...")

        # Створюємо команду FFmpeg для додавання субтитрів
        cmd = [
            'ffmpeg', '-i', video_path, '-vf',
            f"subtitles={subtitles_path}:force_style='FontName={subtitle_style['FontName']},FontSize={subtitle_style['FontSize']},PrimaryColour={subtitle_style['PrimaryColour']},BackColour={subtitle_style['BackColour']},Bold={subtitle_style['Bold']},BorderStyle={subtitle_style['BorderStyle']},Outline={subtitle_style['Outline']},Shadow={subtitle_style['Shadow']},Alignment={subtitle_style['Alignment']},MarginV={subtitle_style['MarginV']}'",
            '-c:a', 'copy', output_path, '-y'
        ]

        result = subprocess.run(cmd, capture_output=True, text=True)

        if result.returncode == 0:
            print(f"✅ Субтитри успішно додані!")
            return output_path
        else:
            print(f"❌ Помилка додавання субтитрів: {result.stderr}")
            return None

    except Exception as e:
        print(f"❌ Помилка: {e}")
        return None

def create_srt_subtitles(subtitles_data, output_path):
    """
    Створює файл субтитрів у форматі SRT
    subtitles_data: список словників з ключами 'start', 'end', 'text'
    """
    try:
        with open(output_path, 'w', encoding='utf-8') as f:
            for i, subtitle in enumerate(subtitles_data, 1):
                # Конвертуємо секунди в формат SRT (00:00:00,000)
                start_time = format_time_srt(subtitle['start'])
                end_time = format_time_srt(subtitle['end'])

                f.write(f"{i}\n")
                f.write(f"{start_time} --> {end_time}\n")
                f.write(f"{subtitle['text']}\n\n")

        print(f"✅ Файл субтитрів створено: {output_path}")
        return output_path
    except Exception as e:
        print(f"❌ Помилка створення субтитрів: {e}")
        return None

def format_time_srt(seconds):
    """Конвертує секунди в формат SRT часу"""
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = int(seconds % 60)
    milliseconds = int((seconds % 1) * 1000)
    return f"{hours:02d}:{minutes:02d}:{secs:02d},{milliseconds:03d}"


In [ ]:
def create_karaoke_ass_subtitles(subtitles_data, output_path, karaoke_style=None):
    """
    Створює файл субтитрів у форматі ASS з ефектом караоке
    """
    if karaoke_style is None:
        karaoke_style = {
            'FontName': 'Arial Black',
            'FontSize': '12',
            'PrimaryColour': '&H00FFFFFF',    # Білий текст (заповнений)
            'SecondaryColour': '&H00808080',  # Сірий текст (незаповнений)
            'OutlineColour': '&H00000000',   # Чорний контур
            'BackColour': '&H80000000',      # Напівпрозорий фон
            'Bold': '1',
            'BorderStyle': '1',
            'Outline': '2',
            'Shadow': '1',
            'Alignment': '2',
            'MarginV': '30'
        }

    try:
        with open(output_path, 'w', encoding='utf-8') as f:
            # Заголовок ASS файлу
            f.write("[Script Info]\n")
            f.write("Title: Karaoke Subtitles\n")
            f.write("ScriptType: v4.00+\n\n")

            # Стилі
            f.write("[V4+ Styles]\n")
            f.write("Format: Name, Fontname, Fontsize, PrimaryColour, SecondaryColour, OutlineColour, BackColour, Bold, Italic, Underline, StrikeOut, ScaleX, ScaleY, Spacing, Angle, BorderStyle, Outline, Shadow, Alignment, MarginL, MarginR, MarginV, Encoding\n")
            f.write(f"Style: Karaoke,{karaoke_style['FontName']},{karaoke_style['FontSize']},{karaoke_style['PrimaryColour']},{karaoke_style['SecondaryColour']},{karaoke_style['OutlineColour']},{karaoke_style['BackColour']},{karaoke_style['Bold']},0,0,0,100,100,0,0,{karaoke_style['BorderStyle']},{karaoke_style['Outline']},{karaoke_style['Shadow']},{karaoke_style['Alignment']},10,10,{karaoke_style['MarginV']},1\n\n")

            # Події
            f.write("[Events]\n")
            f.write("Format: Layer, Start, End, Style, Name, MarginL, MarginR, MarginV, Effect, Text\n")

            for subtitle in subtitles_data:
                start_time = format_time_ass(subtitle['start'])
                end_time = format_time_ass(subtitle['end'])
                text = subtitle['text']

                # Створюємо караоке ефект
                karaoke_text = create_karaoke_effect(text, subtitle['start'], subtitle['end'])

                f.write(f"Dialogue: 0,{start_time},{end_time},Karaoke,,0,0,0,,{karaoke_text}\n")

        print(f"✅ ASS файл з караоке створено: {output_path}")
        return output_path
    except Exception as e:
        print(f"❌ Помилка створення ASS файлу: {e}")
        return None

def format_time_ass(seconds):
    """Конвертує секунди в формат ASS часу"""
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = seconds % 60
    return f"{hours:01d}:{minutes:02d}:{secs:05.2f}"

def create_karaoke_effect(text, start_time, end_time):
    """
    Створює караоке ефект для тексту
    """
    words = text.split()
    if not words:
        return text

    duration = end_time - start_time
    time_per_word = duration / len(words)

    karaoke_text = ""
    current_time = 0

    for i, word in enumerate(words):
        # Тривалість для кожного слова в сантисекундах
        word_duration = int(time_per_word * 100)

        # Додаємо караоке тег для кожного слова
        karaoke_text += f"{{\\k{word_duration}}}{word}"

        if i < len(words) - 1:
            karaoke_text += " "

    return karaoke_text


In [ ]:
def add_karaoke_subtitles(video_path, subtitles_path, output_path, is_ass=True):
    """
    Додає караоке субтитри до відео
    """
    try:
        print(f"🎤 Додавання караоке субтитрів...")

        if is_ass:
            # Для ASS файлів з караоке ефектами
            cmd = [
                'ffmpeg', '-i', video_path,
                '-vf', f"ass={subtitles_path}",
                '-c:a', 'copy', output_path, '-y'
            ]
        else:
            # Для звичайних SRT з стилізацією
            cmd = [
                'ffmpeg', '-i', video_path,
                '-vf', f"subtitles={subtitles_path}:force_style='FontName=Arial Black,FontSize=28,PrimaryColour=&H00FFFF,SecondaryColour=&H808080,Bold=1,BorderStyle=1,Outline=2,Shadow=1,Alignment=2,MarginV=30'",
                '-c:a', 'copy', output_path, '-y'
            ]

        result = subprocess.run(cmd, capture_output=True, text=True)

        if result.returncode == 0:
            print(f"✅ Караоке субтитри успішно додані!")
            return output_path
        else:
            print(f"❌ Помилка додавання караоке: {result.stderr}")
            return None

    except Exception as e:
        print(f"❌ Помилка: {e}")
        return None


In [ ]:
def merge_videos_with_music_and_subtitles(video_urls, music_path, subtitles_data=None, output_filename="final_video.mp4"):
    """
    Склеює відео, накладає музику та додає субтитри
    """
    downloaded_videos = []
    temp_files = []

    try:
        # Створюємо робочу директорію
        work_dir = '/content/video_work'
        os.makedirs(work_dir, exist_ok=True)

        # Завантажуємо всі відео
        print("📥 Завантаження відео...")
        for i, url in enumerate(video_urls):
            video_filename = f"{work_dir}/video_{i+1}.mp4"
            result = download_video_from_url(url, video_filename)

            if result:
                downloaded_videos.append(result)
                print(f"✅ Відео {i+1} готове")
            else:
                print(f"❌ Не вдалося завантажити відео {i+1}")

        if not downloaded_videos:
            print("❌ Не вдалося завантажити жодного відео!")
            return None

        # Створюємо файл списку для ffmpeg concat
        concat_file = f"{work_dir}/concat_list.txt"
        with open(concat_file, 'w') as f:
            for video in downloaded_videos:
                f.write(f"file '{video}'\n")

        temp_files.append(concat_file)

        # Склеюємо відео (без звуку спочатку)
        print("🔗 Склеювання відео...")
        merged_video_no_audio = f"{work_dir}/merged_no_audio.mp4"

        cmd_concat = [
            'ffmpeg', '-f', 'concat', '-safe', '0', '-i', concat_file,
            '-c', 'copy', '-an', merged_video_no_audio, '-y'
        ]

        result = subprocess.run(cmd_concat, capture_output=True, text=True)
        if result.returncode != 0:
            print(f"❌ Помилка склеювання: {result.stderr}")
            return None

        temp_files.append(merged_video_no_audio)

        # Отримуємо тривалість відео
        video_info = get_video_info(merged_video_no_audio)
        if not video_info:
            print("❌ Не вдалося отримати інформацію про відео")
            return None

        video_duration = video_info['duration']
        print(f"📊 Тривалість відео: {video_duration:.2f} секунд")

        # Додаємо субтитри якщо вони є
        current_video = merged_video_no_audio
        if subtitles_data:
            print("📝 Додавання субтитрів...")
            subtitles_file = f"{work_dir}/subtitles.srt"
            create_srt_subtitles(subtitles_data, subtitles_file)
            temp_files.append(subtitles_file)

            video_with_subtitles = f"{work_dir}/video_with_subtitles.mp4"
            result_sub = add_hardcoded_subtitles(merged_video_no_audio, subtitles_file, video_with_subtitles)

            if result_sub:
                current_video = video_with_subtitles
                temp_files.append(video_with_subtitles)
            else:
                print("⚠️ Продовжуємо без субтитрів")

        # Підготовляємо музику
        print("🎵 Підготовка музики...")
        prepared_audio = f"{work_dir}/prepared_audio.mp3"

        cmd_audio = [
            'ffmpeg', '-stream_loop', '-1', '-i', music_path,
            '-t', str(video_duration), '-acodec', 'libmp3lame',
            '-ab', '128k', prepared_audio, '-y'
        ]

        result = subprocess.run(cmd_audio, capture_output=True, text=True)
        if result.returncode != 0:
            print(f"❌ Помилка підготовки аудіо: {result.stderr}")
            return None

        temp_files.append(prepared_audio)

        # Фінальне з'єднання відео та музики
        print("🎬 Створення фінального відео...")
        final_output = f"/content/{output_filename}"

        cmd_final = [
            'ffmpeg', '-i', current_video, '-i', prepared_audio,
            '-c:v', 'libx264', '-c:a', 'aac', '-strict', 'experimental',
            '-shortest', final_output, '-y'
        ]

        result = subprocess.run(cmd_final, capture_output=True, text=True)

        if result.returncode != 0:
            print(f"❌ Помилка створення фінального відео: {result.stderr}")
            return None

        # Інформація про фінальний файл
        if os.path.exists(final_output):
            file_size = os.path.getsize(final_output) / (1024*1024)
            final_info = get_video_info(final_output)

            print(f"✅ Відео успішно створено!")
            print(f"📁 Файл: {final_output}")
            print(f"📊 Розмір: {file_size:.2f} MB")
            if final_info:
                print(f"⏱️ Тривалість: {final_info['duration']:.2f} сек")
                print(f"🎥 Роздільність: {final_info['width']}x{final_info['height']}")

            return final_output
        else:
            print("❌ Фінальний файл не створено")
            return None

    except Exception as e:
        print(f"❌ Критична помилка: {e}")
        return None

    finally:
        # Очищення тимчасових файлів
        print("🧹 Очищення тимчасових файлів...")
        for temp_file in temp_files + downloaded_videos:
            try:
                if os.path.exists(temp_file):
                    os.remove(temp_file)
            except:
                pass

        try:
            if os.path.exists(work_dir):
                import shutil
                shutil.rmtree(work_dir)
        except:
            pass


In [ ]:
def merge_videos_with_karaoke_subtitles(video_urls, music_path, karaoke_data=None, output_filename="karaoke_video.mp4"):
    """
    Основна функція з підтримкою караоке ефектів
    """
    downloaded_videos = []
    temp_files = []

    try:
        work_dir = '/content/video_work'
        os.makedirs(work_dir, exist_ok=True)

        # Завантажуємо та склеюємо відео (як раніше)
        print("📥 Завантаження відео...")
        for i, url in enumerate(video_urls):
            video_filename = f"{work_dir}/video_{i+1}.mp4"
            result = download_video_from_url(url, video_filename)

            if result:
                downloaded_videos.append(result)
                print(f"✅ Відео {i+1} готове")
            else:
                print(f"❌ Не вдалося завантажити відео {i+1}")

        if not downloaded_videos:
            print("❌ Не вдалося завантажити жодного відео!")
            return None

        # Створюємо файл списку для ffmpeg concat
        concat_file = f"{work_dir}/concat_list.txt"
        with open(concat_file, 'w') as f:
            for video in downloaded_videos:
                f.write(f"file '{video}'\n")

        temp_files.append(concat_file)

        # Склеюємо відео (без звуку спочатку)
        print("🔗 Склеювання відео...")
        merged_video_no_audio = f"{work_dir}/merged_no_audio.mp4"

        cmd_concat = [
            'ffmpeg', '-f', 'concat', '-safe', '0', '-i', concat_file,
            '-c', 'copy', '-an', merged_video_no_audio, '-y'
        ]

        result = subprocess.run(cmd_concat, capture_output=True, text=True)
        if result.returncode != 0:
            print(f"❌ Помилка склеювання: {result.stderr}")
            return None

        temp_files.append(merged_video_no_audio)

        # Отримуємо тривалість відео
        video_info = get_video_info(merged_video_no_audio)
        if not video_info:
            print("❌ Не вдалося отримати інформацію про відео")
            return None

        video_duration = video_info['duration']
        print(f"📊 Тривалість відео: {video_duration:.2f} секунд")

        # Додаємо караоке ефект якщо є дані
        current_video = merged_video_no_audio  # після склеювання відео

        if karaoke_data:
            print("🎤 Створення караоке ефекту...")

            # Створюємо ASS файл з караоке
            ass_file = f"{work_dir}/karaoke.ass"
            create_karaoke_ass_subtitles(karaoke_data, ass_file)
            temp_files.append(ass_file)

            # Додаємо караоке до відео
            video_with_karaoke = f"{work_dir}/video_with_karaoke.mp4"
            result_karaoke = add_karaoke_subtitles(current_video, ass_file, video_with_karaoke, is_ass=True)

            if result_karaoke:
                current_video = video_with_karaoke
                temp_files.append(video_with_karaoke)
            else:
                print("⚠️ Продовжуємо без караоке ефекту")

        # Додаємо музику та завершуємо (як раніше)
        print("🎵 Підготовка музики...")
        prepared_audio = f"{work_dir}/prepared_audio.mp3"

        cmd_audio = [
            'ffmpeg', '-stream_loop', '-1', '-i', music_path,
            '-t', str(video_duration), '-acodec', 'libmp3lame',
            '-ab', '128k', prepared_audio, '-y'
        ]

        result = subprocess.run(cmd_audio, capture_output=True, text=True)
        if result.returncode != 0:
            print(f"❌ Помилка підготовки аудіо: {result.stderr}")
            return None

        temp_files.append(prepared_audio)

        # Фінальне з'єднання відео та музики
        print("🎬 Створення фінального відео...")
        final_output = f"/content/{output_filename}"

        cmd_final = [
            'ffmpeg', '-i', current_video, '-i', prepared_audio,
            '-c:v', 'libx264', '-c:a', 'aac', '-strict', 'experimental',
            '-shortest', final_output, '-y'
        ]

        result = subprocess.run(cmd_final, capture_output=True, text=True)

        if result.returncode != 0:
            print(f"❌ Помилка створення фінального відео: {result.stderr}")
            return None

        # Інформація про фінальний файл
        if os.path.exists(final_output):
            file_size = os.path.getsize(final_output) / (1024*1024)
            final_info = get_video_info(final_output)

            print(f"✅ Відео успішно створено!")
            print(f"📁 Файл: {final_output}")
            print(f"📊 Розмір: {file_size:.2f} MB")
            if final_info:
                print(f"⏱️ Тривалість: {final_info['duration']:.2f} сек")
                print(f"🎥 Роздільність: {final_info['width']}x{final_info['height']}")

            return final_output
        else:
            print("❌ Фінальний файл не створено")
            return None

    except Exception as e:
        print(f"❌ Критична помилка: {e}")
        return None
    finally:
        # Очищення файлів
        pass


In [ ]:
# ========== НАЛАШТУВАННЯ ==========

# 1. Прямі посилання на відео (замініть на ваші)
video_urls = [
  "https://video-product.cdn.minimax.io/inference_output/video/2025-08-29/93f99dbd-9614-411c-b450-b2e6972bfb19/output.mp4",
  "https://video-product.cdn.minimax.io/inference_output/video/2025-08-29/e68ef947-0dc5-442c-9690-365fe19c72e5/output.mp4",
  "https://video-product.cdn.minimax.io/inference_output/video/2025-08-29/bda5961e-0bef-473b-9ee6-a6fac4d8cc6d/output.mp4",
  "https://video-product.cdn.minimax.io/inference_output/video/2025-08-29/b116cdbf-2ca5-49c5-8dc3-b8528b7e03cb/output.mp4",
  "https://video-product.cdn.minimax.io/inference_output/video/2025-08-29/bfd6a679-b950-434f-8152-9d4f173e0521/output.mp4",
  "https://video-product.cdn.minimax.io/inference_output/video/2025-08-29/6eae3863-2eb2-4e7d-8d52-fef953892b74/output.mp4",
  "https://video-product.cdn.minimax.io/inference_output/video/2025-08-29/ea130753-7c14-44f8-a9c6-4bb2bca3130e/output.mp4",
  "https://video-product.cdn.minimax.io/inference_output/video/2025-08-29/081eebcd-50e9-4c27-b602-910ef1d15457/output.mp4",
  "https://video-product.cdn.minimax.io/inference_output/video/2025-08-29/f3069ac3-ca53-4b5a-98a6-010e5a7274b6/output.mp4",
  "https://video-product.cdn.minimax.io/inference_output/video/2025-08-29/72c41b26-bab7-41fe-b3f9-34ac35560f4e/output.mp4",
  "https://video-product.cdn.minimax.io/inference_output/video/2025-08-29/2e0b1ad7-1755-4c1a-a63e-fb321609ed69/output.mp4",
  "https://video-product.cdn.minimax.io/inference_output/video/2025-08-29/ce4184a6-426b-470a-95e6-43c3a050b52d/output.mp4"
    # Додайте скільки потрібно посилань
]

# 2. Шлях до музики на Google Drive
music_path = "/content/drive/MyDrive/Generated video/music/1 Фісташки (16_44).wav"  # Змініть на ваш шлях

subtitles_example = [
      {
        "start": 13.0,
        "end": 19.0,
        "text": "Я віддавав себе всього роботі, я відкладав особисте на потім"
      },
      {
        "start": 19.0,
        "end": 25.0,
        "text": "А ти кричала, била посуд, просила увімкнути серце і вимкнути мозок"
      },
      {
        "start": 25.0,
        "end": 31.0,
        "text": "А я не вмію так жити, ти ж знаєш, сама обрала, за кого виходила заміж"
      },
      {
        "start": 31.0,
        "end": 37.0,
        "text": " Я все ще той, але уже трішки інакший, всі мої фішки поміняю на твої фісташки"
      },
      {
        "start": 37.6,
        "end": 44.0,
        "text": " А твої сльози пали на подушки, бо в нас не так, як пишуть твої книжки"
      },
      {
        "start": 44.0,
        "end": 49.0,
        "text": " Але обійми мої наче дужки, а в них слово кохаю"
      },
      {
        "start": 49.0,
        "end": 55.0,
        "text": " Йдеш? — то іди, і не обертайся, в добру дорогу відпущу"
      },
      {
        "start": 55.0,
        "end": 61.0,
        "text": " Як стане важко, не повертайся, тому що досі тебе люблю"
      },
      {
        "start": 61.0,
        "end": 67.0,
        "text": " Йдеш? — то іди, і не обертайся, в добру дорогу відпущу"
      },
      {
        "start": 67.0,
        "end": 74.0,
        "text": " Як стане важко, не повертайся, тому що знову тебе люблю"
      }

    ]

# 3. Назва вихідного файлу
output_filename = "my_merged_video_ffmpeg.mp4"

# ========== ПЕРЕВІРКИ ==========

print("🔍 Перевірка налаштувань...")

# Перевіряємо ffmpeg
result = subprocess.run(['ffmpeg', '-version'], capture_output=True, text=True)
if result.returncode == 0:
    print("✅ FFmpeg готовий до роботи")
else:
    print("❌ Проблема з FFmpeg")

# Перевіряємо музику
if os.path.exists(music_path):
    print(f"✅ Музичний файл знайдено: {music_path}")

    # Отримуємо інфо про музику
    music_info = get_video_info(music_path)
    if music_info:
        print(f"🎵 Тривалість музики: {music_info['duration']:.2f} сек")
else:
    print(f"❌ Музичний файл не знайдено: {music_path}")
    print("Будь ласка, перевірте шлях до файлу")

print(f"📹 Кількість відео для склеювання: {len(video_urls)}")

# ========== ЗАПУСК ==========

if os.path.exists(music_path) and len(video_urls) > 0:
    print("\n🚀 Запускаємо обробку...")
    result_path = merge_videos_with_karaoke_subtitles(video_urls, music_path, subtitles_example, output_filename)

    if result_path:
        print(f"\n🎉 ГОТОВО! Відео створено: {result_path}")

        # Автоматично завантажуємо файл
        try:
            files.download(result_path)
            print("📥 Файл завантажується...")
        except Exception as e:
            print(f"Помилка завантаження: {e}")
            print("Ви можете знайти файл за шляхом:", result_path)
    else:
        print("\n❌ Не вдалося створити відео")
else:
    print("\n❌ Перевірте налаштування перед запуском")


🔍 Перевірка налаштувань...
✅ FFmpeg готовий до роботи
✅ Музичний файл знайдено: /content/drive/MyDrive/Generated video/music/1 Фісташки (16_44).wav
🎵 Тривалість музики: 138.46 сек
📹 Кількість відео для склеювання: 12

🚀 Запускаємо обробку...
📥 Завантаження відео...
Завантаження відео: /content/video_work/video_1.mp4
✅ Відео завантажено: /content/video_work/video_1.mp4
✅ Відео 1 готове
Завантаження відео: /content/video_work/video_2.mp4
✅ Відео завантажено: /content/video_work/video_2.mp4
✅ Відео 2 готове
Завантаження відео: /content/video_work/video_3.mp4
✅ Відео завантажено: /content/video_work/video_3.mp4
✅ Відео 3 готове
Завантаження відео: /content/video_work/video_4.mp4
✅ Відео завантажено: /content/video_work/video_4.mp4
✅ Відео 4 готове
Завантаження відео: /content/video_work/video_5.mp4
✅ Відео завантажено: /content/video_work/video_5.mp4
✅ Відео 5 готове
Завантаження відео: /content/video_work/video_6.mp4
✅ Відео завантажено: /content/video_work/video_6.mp4
✅ Відео 6 готове
За

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

📥 Файл завантажується...


In [ ]:
# Test code snippet to verify logic before applying to the main codebase
# Note: I am not running this here, just using it as scratchpad or for user to see if requested.
# I will proceed with editing the actual python file.
pass